# Multiscale MILP

This is a continuation of 'One Location, One Temporal Scale, One Operation, Linear Programming Example' [Example 1]. Refer Example 1 to learn the basics on how Energia models processes. 

In this example, we add another Process [Solar PV] and Storage [Li-ion Battery]. Technology choice is modeled using binaries. Moreover, the model is multiscale as the operational capacities are decision variables. 

In [1]:
from energia import *

m = Model('example2')
m.q = Periods()
m.y = 4 * m.q
m.usd = Currency()

In [2]:
-m.q

-1q

## Resources

### Conveniently declaring components

Use m.declare(\<Object Type\>, \<list of names\>) to declare a large number of objects in one step.

In [19]:
m.declare(Resource, ['power', 'wind', 'solar'])

### Set bounds on Resource flows

Unlike wind which has bound on the total consumption, we set a daily limit on solar energy. The same bound is repeated in each quarter. The following constraints are written.

$\mathbf{cons}_{solar, network, quarter_0} \leq 100$

$\mathbf{cons}_{solar, network, quarter_1} \leq 100$

$\mathbf{cons}_{solar, network, quarter_2} \leq 100$

$\mathbf{cons}_{solar, network, quarter_3} \leq 100$

In [20]:
m.solar.consume(m.q) <= 100
m.wind.consume <= 400
m.power.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]

2025-10-23 12:53:01,085 [INFO] Balance for solar in (l0, q): initializing
2025-10-23 12:53:01,086 [INFO] ✔ Completed in 0.0001506805419921875 seconds
2025-10-23 12:53:01,086 [INFO] Binding consume in domain (solar, l0, q)
2025-10-23 12:53:01,087 [INFO] ✔ Completed in 0.00011897087097167969 seconds
2025-10-23 12:53:01,087 [INFO] Balance for wind in (l0, y): initializing
2025-10-23 12:53:01,088 [INFO] ✔ Completed in 9.751319885253906e-05 seconds
2025-10-23 12:53:01,088 [INFO] Binding consume in domain (wind, l0, y)
2025-10-23 12:53:01,088 [INFO] ✔ Completed in 6.699562072753906e-05 seconds
2025-10-23 12:53:01,089 [INFO] Balance for power in (l0, q): initializing
2025-10-23 12:53:01,089 [INFO] ✔ Completed in 0.00012159347534179688 seconds
2025-10-23 12:53:01,090 [INFO] Binding release in domain (power, l0, q)
2025-10-23 12:53:01,090 [INFO] ✔ Completed in 0.00037407875061035156 seconds


## Operations 

### Capacity as a variable 

Here we want the optimization problem to determine the optimal capacity. Moreover, we set binaries to avoid the lower bound being adhered to if the process is not set up. 

If the bounds are meant to be compulsory limits, skip the .x 

In [21]:
m.wf = Process()
m.wf(m.power) == -1 * m.wind
m.wf.capacity.x <= 100
m.wf.capacity.x >= 10
m.capacity.show()

2025-10-23 12:53:01,096 [INFO] Binding capacity in domain (wf, l0, y)
2025-10-23 12:53:01,096 [INFO] ✔ Completed in 0.0001895427703857422 seconds
2025-10-23 12:53:01,097 [INFO] Binding capacity in domain (wf, l0, y)
2025-10-23 12:53:01,097 [INFO] ✔ Completed in 0.00016307830810546875 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

Unlike in Example 1, where the capacity was know, capacity is a variable here. 

Moreover, the expenditure associated with operating and capacitating are different

In [22]:
m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]
m.wf.capacity[m.usd.spend] == 990637 + 3354
m.wf.operate[m.usd.spend] == 49
m.operate.show(True)

2025-10-23 12:53:01,104 [INFO] Binding operate in domain (wf, l0, q)
2025-10-23 12:53:01,104 [INFO] ✔ Completed in 0.0002281665802001953 seconds
2025-10-23 12:53:01,105 [INFO] Mapping operate: (wf, l0, q) → (wf, l0, y)
2025-10-23 12:53:01,106 [INFO] ✔ Completed in 0.00015544891357421875 seconds


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [23]:
m.pv = Process()
m.pv(m.power) == -1 * m.solar
m.pv.capacity.x <= 100
m.pv.capacity.x >= 10

m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
m.pv.capacity[m.usd.spend] == 567000 + 872046
m.pv.operate[m.usd.spend] == 90000

2025-10-23 12:53:01,116 [INFO] Binding capacity in domain (pv, l0, y)
2025-10-23 12:53:01,116 [INFO] ✔ Completed in 0.00020933151245117188 seconds
2025-10-23 12:53:01,117 [INFO] Binding capacity in domain (pv, l0, y)
2025-10-23 12:53:01,117 [INFO] ✔ Completed in 0.0002613067626953125 seconds
2025-10-23 12:53:01,118 [INFO] Binding operate in domain (pv, l0, q)
2025-10-23 12:53:01,119 [INFO] ✔ Completed in 0.00022411346435546875 seconds
2025-10-23 12:53:01,120 [INFO] Mapping operate: (pv, l0, q) → (pv, l0, y)
2025-10-23 12:53:01,121 [INFO] ✔ Completed in 0.00011181831359863281 seconds


### Storage Operation

energia now allows storing to require the use of other resources, example power for hydrogen cryogenic storage. 

Provide an equation similar to Process, in this case the basis is the stored resource 
If no other resource is provided, it is assumed to be the charging/discharging efficiency

Note that the following are created internally: 
1. auxilary resource  with name resource.stored 
2. charging and discharging processes as storage.charge and storage.discharge 

The parameters for each of these can be set individually, thus allowing for a wide range of modeling approaches 

In [24]:
m.lii = Storage()

m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10

# m.lii.capacity >= 10
m.lii.capacity[m.usd.spend] == 1302182 + 41432

m.lii.inventory[m.usd.spend] == 2000


2025-10-23 12:53:01,126 [INFO] Binding invcapacity in domain (lii.stored, l0, y)
2025-10-23 12:53:01,127 [INFO] ✔ Completed in 0.00015616416931152344 seconds
2025-10-23 12:53:01,127 [INFO] Binding invcapacity in domain (lii.stored, l0, y)
2025-10-23 12:53:01,128 [INFO] ✔ Completed in 0.0001709461212158203 seconds
2025-10-23 12:53:01,129 [INFO] Balance for lii.stored in (l0, y): initializing


aaa η(lii.discharge, None)


## Locating Operations

Operations can be located as 

operation.locate(\<list of locations\>)

or 

m.location.operations(\<list of operations\>)

They both do the same thing 

In [25]:
m.pem = Process()

m.pem(-m.power) == m.solar 

η(pem, power)

In [26]:
m.pv.locate(m.network)
m.network.locate(m.wf, m.lii)

2025-10-23 12:53:01,140 [INFO] Balance for power in (l0, q): adding produce(power, l0, q, operate, pv)
2025-10-23 12:53:01,141 [INFO] ✔ Completed in 0.00017786026000976562 seconds
2025-10-23 12:53:01,142 [INFO] Balance for solar in (l0, q): adding expend(solar, l0, q, operate, pv)
2025-10-23 12:53:01,142 [INFO] ✔ Completed in 0.00014781951904296875 seconds
2025-10-23 12:53:01,143 [INFO] Balance for power in (l0, q): adding produce(power, l0, q, operate, wf)
2025-10-23 12:53:01,144 [INFO] ✔ Completed in 0.000152587890625 seconds
2025-10-23 12:53:01,144 [INFO] Balance for wind in (l0, y): adding expend(wind, l0, y, operate, wf)
2025-10-23 12:53:01,145 [INFO] ✔ Completed in 8.440017700195312e-05 seconds
2025-10-23 12:53:01,145 [INFO] Assuming  lii.stored inventory capacity is unbounded in (l0, y)
2025-10-23 12:53:01,145 [INFO] Assuming inventory of lii.stored is unbounded in (l0, y)
2025-10-23 12:53:01,145 [INFO] Assuming inventory of lii.stored is bound by inventory capacity in (l0, q)
2

## Inventory Balance

Inventory is passed on from one time period (t - 1) to the next (t) and hence features in the general resource balance for resource.stored 

In [27]:
m.inventory.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Optimize!

In [28]:
m.usd.spend.opt()

2025-10-23 12:53:01,170 [INFO] Mapping spend: (usd, l0, y, capacity, wf) → (usd, l0, y)
2025-10-23 12:53:01,171 [INFO] ✔ Completed in 0.0001628398895263672 seconds
2025-10-23 12:53:01,171 [INFO] Mapping spend: (usd, l0, y, operate, wf) → (usd, l0, y)
2025-10-23 12:53:01,172 [INFO] ✔ Completed in 0.00013399124145507812 seconds
2025-10-23 12:53:01,172 [INFO] Mapping spend: (usd, l0, y, capacity, pv) → (usd, l0, y)
2025-10-23 12:53:01,173 [INFO] ✔ Completed in 0.00011324882507324219 seconds
2025-10-23 12:53:01,173 [INFO] Mapping spend: (usd, l0, y, operate, pv) → (usd, l0, y)
2025-10-23 12:53:01,174 [INFO] ✔ Completed in 0.0001246929168701172 seconds
2025-10-23 12:53:01,174 [INFO] Mapping spend: (usd, l0, y, invcapacity, lii.stored) → (usd, l0, y)
2025-10-23 12:53:01,175 [INFO] ✔ Completed in 0.00013780593872070312 seconds
2025-10-23 12:53:01,175 [INFO] Mapping spend: (usd, l0, y, inventory, lii.stored) → (usd, l0, y)
2025-10-23 12:53:01,176 [INFO] ✔ Completed in 0.00011277198791503906 se

Read MPS format model from file Program(example2).mps
Reading time = 0.00 seconds
PROGRAM(EXAMPLE2): 83 rows, 78 columns, 196 nonzeros


2025-10-23 12:53:01,183 [INFO] Optimizing Program(example2) using gurobi


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 83 rows, 78 columns and 196 nonzeros
Model fingerprint: 0xc4ff6a31
Variable types: 75 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [6e-01, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [7e+01, 4e+02]
Presolve removed 71 rows and 66 columns
Presolve time: 0.00s
Presolved: 12 rows, 12 columns, 34 nonzeros
Variable types: 12 continuous, 0 integer (0 binary)

Root relaxation: objective 3.006497e+08, 6 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    3.006497e+08 3.0065e+08  0.0

2025-10-23 12:53:01,197 [INFO] Solution found. Use .output() to display it
2025-10-23 12:53:01,198 [INFO] Creating Solution object, check.solution


## Solution

### Inventory Profiles

The inventory maintained in each time period is:

In [29]:
m.inventory.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount charged into inventory is:

In [30]:
m.produce(m.power.lii, m.lii.charge.operate, m.q).output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

The amount discharged from inventory is:

In [31]:
m.produce(m.power, m.lii.discharge.operate, m.q).output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Integer Decisions 

All the operations are setup in this case

In [32]:
m.capacity.reporting.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [33]:
m.capacity.output()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [34]:
m.show(True)

# Mathematical Program for Program(example2)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>